In [1]:
from url_delect import parse_url
import pandas as pd
import re
import numpy as np


def df_set():
    f = pd.read_csv('D:\\luzheng\\桌面\\action_log1601.csv', sep='--->', 
                header = None, engine='python', iterator = True,
                names = ['uuid', 'uid', 'ref', 'url', 'time', 'ip'])
    df = f.get_chunk(300)
    return df


def reserve_time():
    df = df_set()
    r_time = {}
    for k, v in df.groupby([df['uuid'], df['time']]):
        for url in v['url']:
            url = parse_url(url)
            if re.match(r'.*reserve', url):
                r_time[k[0]] = r_time.get(k[0], []) + [k[1]]
    return r_time


def first_time():
    df = df_set()
    reserve_data = reserve_time()
    f_time = {}
    for k, v in df.groupby(df['uuid']):
        if k in reserve_data.keys():
            f_time[k] = [v['time'].tolist()[0],
                         reserve_data.get(k)[0]]
    return f_time


def averge_time(data):
    time = np.array([data.get(uuid)[1] - data.get(uuid)[0]
                     for uuid in data])
    avg_time = time.sum()
    return avg_time / time.shape[0]


data_set = first_time()
averge_time(data_set)

4.3888888888888893

In [54]:
from ncode import NCode
from sklearn import neighbors
from sklearn import cross_validation
import pymysql
import pandas as pd
import numpy as np


def df_set():
    f = pd.read_csv('D:\\luzheng\\桌面\\action_log1601.csv', sep='--->', 
                header = None, engine='python', iterator = True,
                names = ['uuid', 'uid', 'ref', 'url', 'time', 'ip'])
    df = f.get_chunk(10000)
    return df


def paid_id(uid):
    conn = pymysql.connect(host='***',port=3306,user='***',
                           passwd='***',db='talk',charset='utf8')
    cur = conn.cursor()
    cur.execute('''select id,is_buy from user where id = '''+uid+'''
                ''')
    paid_uid = [col[1] for col in cur]
    return paid_uid
    

def dataset():
    df = df_set()
    uid = []
    for k, v in df.groupby(df['uid']):
        k = NCode().decode(k)
        time = v['time'].tolist()[-1] - v['time'].tolist()[0]
        if paid_id(k)[0] == 'buy':
            uid.append([time, 1])
        else:
            uid.append([time, 0])
    return np.array(uid)


x = dataset()[:, :1]
y = dataset()[:, 1]
scores = []
rs = cross_validation.ShuffleSplit(4, n_iter=3, test_size=.25, random_state=0)
for train, test in rs:
    x_train, x_test = x[train], x[test]
    y_train, y_test = y[train], y[test]
    model = neighbors.KNeighborsClassifier()
    model.fit(x, y)
    scores.append(model.score(x_test, y_test))
print(np.mean(scores), np.std(scores))

0.666666666667 0.471404520791


In [78]:
from sklearn import neighbors
from sklearn import cross_validation
import pandas as pd
import numpy as np


def df_set():
    f = pd.read_csv('D:\\luzheng\\桌面\\page_step.csv', sep='|', 
                header = None, iterator = True,
                names = ['UserID', 'ClickDate', 'TimeOnPage', 'VisitStepID'])
    df = f.get_chunk(30000)
    return df


def user_data():
    df = df_set()
    time = []
    for k, v in df.groupby('UserID'):
        if k == 0: continue
        mean_time = np.mean(v['TimeOnPage'].tolist())
        if mean_time == -1: continue
            
        if mean_time < 100:
            time.append([mean_time, 1])
        else:
            time.append([mean_time, 0])
    return np.array(time)


x = user_data()[:, :1]
y = user_data()[:, 1]
scores = []
rs = cross_validation.ShuffleSplit(4, n_iter=3, test_size=.25, random_state=0)
for train, test in rs:
    x_train, x_test = x[train], x[test]
    y_train, y_test = y[train], y[test]
    clf = neighbors.KNeighborsClassifier()
    clf.fit(x, y)
    scores.append(clf.score(x_test, y_test))
print(np.mean(scores), np.std(scores))

1.0 0.0


In [33]:
import pandas as pd
import pymysql
import re
from datetime import datetime,date,timedelta

#user_analysis

def df_set():
    f = pd.read_csv('D:\\luzheng\\桌面\\page_step_new_2.csv', sep='*', 
                header = None, iterator = True, encoding = 'latin1', dtype='unicode',
                names = ['uid', 'ref', 'url', 'click_date', 'time_page', 'step'])
    df = f.get_chunk(14659399) #total rows: 14659399
    df[['time_page', 'step']] = df[['time_page', 'step']].astype(float)
    df['time_page'] = df['time_page'].replace(-1, 0)
    return df


def df_teacher(df):
    pattern_teacher = re.compile(r'.*/teacher/[a-z]+/[a-z0-9]')
    #pattern = re.compile(r'.*/reserve/')
    #pattern_tool = re.compile(r'.*/trial/tool')
    #pattern_product = re.compile(r'.*/pay/product')
    #pattern_pay = re.compile(r'.*/pay/(?:order|orderPay)')
    #pattern_index_www = re.compile(r'.*://(www\.51talk\.(com|cn|net))($|/landing)')
    #pattern_index_wap = re.compile(r'.*://(wap\.51talk\.(com|cn|net))($|/landing)')
    #pattern_user_info = re.compile(r'.*/user/info')
    type_url = []
    for url in df['url']:
        if pattern_teacher.match(url):
            type_url.append('teacher')
        #elif pattern_tool.match(url):
        #    type_url.append('tool')
        #elif pattern_product.match(url):
        #    type_url.append('product')
        #elif pattern_pay.match(url):
        #    type_url.append('pay')
        #elif pattern_index_www.match(url):
        #    type_url.append('index_www')
        #elif pattern_index_wap.match(url):
        #    type_url.append('index_wap')
        #elif pattern_user_info.match(url):
        #    type_url.append('user_info')   
        else:
            type_url.append('--')            
    df['type_url'] = type_url
    df_teacher = df[df['type_url']=='teacher']
    return df_teacher


def search_t(df):
    df = df.copy()
    teacher_list = []
    pattern = re.compile(r'.*/teacher/[a-z]+/([a-z0-9]+)')
    for index, row in df.iterrows():
        m = pattern.match(row['url'])
        teacher_list.append(m.group(1))
    df['t_id'] = teacher_list
    return df


def resault_student(df):
    grouped = df.groupby('uid')
    step_data = [[k, v['step'].sum()] for k, v in grouped]
    time_page_data = [[k, v['time_page'].sum()] for k, v in grouped]
    
    resault1 = pd.DataFrame(step_data, columns=['uid', 'step_total'])
    resault2 = pd.DataFrame(time_page_data, columns=['uid', 'time_total'])
    resault = pd.merge(resault1, resault2, how='outer', on='uid')
    return resault
    #resault.to_csv('D:/luzheng/桌面/student_page_time.csv')

    
def student_dis(df):
    uid = df['uid'].tolist()
    conn = pymysql.connect(host='***',port=3306,user='***',
                           passwd='***',db='talk',charset='utf8')
    cur = conn.cursor(pymysql.cursors.SSCursor)
    cur.execute('''select id, sex, age, is_buy, city, is_staff, birthday from user
                   where id in ('''+",".join(uid)+''')
                ''')
    user_data = [[str(row[0]), row[1], row[2], 
                  row[3], row[4], row[5], row[6]] for row in cur]
        
    df_user = pd.DataFrame(user_data, columns=['uid', 'sex', 'age', 
                                               'buy_type', 'city','staff_type','birthday'])
    df_stu = pd.merge(df, df_user, how='left', on='uid')
    
    df_stu['age'].fillna(0, inplace = True)
    df_stu['sex'].fillna('-', inplace = True)
    df_stu['city'].fillna('未知', inplace = True)
    df_stu['city'] = df_stu['city'].replace('', '未知')
    df_stu['birthday'] = pd.to_datetime(df_stu['birthday'], errors='coerce')
    df_stu['age_new'] = pd.datetime.now().year - df_stu['birthday'].dt.year
    
    return df_stu
    #df_stu.to_csv('D:/luzheng/桌面/student_data.csv', encoding='utf-8')
    

def resault_teacher(df):
    grouped = df.groupby('t_id')
    data = []
    for k, v in grouped:
        stu_count = len(set(v['uid'].tolist()))
        total_time = v['time_page'].sum()
        
        data.append([k, stu_count, total_time])
        resault = pd.DataFrame(data, columns=['t_id', 'stu_count', 'page_time'])
        resault.sort_values('page_time', ascending=0).to_csv('D:/luzheng/桌面/teacher_page_time.csv')

        
#teacher_resault
df_t = df_teacher(df_set())
df = search_t(df_t)
resault_teacher(df)

#student_resault
#resault_student(df_set())
#student_dis(resault_student(df_set()))


#count_stu
#df.groupby(['t_id', 'uid']).sum().sort('time_page').to_csv('D:/luzheng/桌面/teacher_page_time_2.csv')

#piovt_table
#pd.pivot_table(df, index = ['url', 'uid', 'click_date'],
#                values = ['time_page'], aggfunc = [np.sum], fill_value = 0)

In [4]:
import pandas as pd
import pymysql
import re
from datetime import datetime,date,timedelta

#user_analysis_student

def df_set():
    f = pd.read_csv('D:\\luzheng\\桌面\\page_step_new_3.csv', sep='*', 
                header = None, iterator = True, encoding = 'latin1', dtype='unicode',
                names = ['uid', 'ref', 'url', 'click_date','click_date_time', 'time_page', 'step'])
    df = f.get_chunk(201070) #total rows: 201070
    df[['time_page', 'step']] = df[['time_page', 'step']].astype(float)
    df['time_page'] = df['time_page'].replace(-1, 0)
    return df


def resault_student(df):
    grouped = df.groupby('uid')
    step_data = [[k, v['step'].sum()] for k, v in grouped]
    time_page_data = [[k, v['time_page'].sum()] for k, v in grouped]
    
    resault1 = pd.DataFrame(step_data, columns=['uid', 'step_total'])
    resault2 = pd.DataFrame(time_page_data, columns=['uid', 'time_total'])
    resault = pd.merge(resault1, resault2, how='outer', on='uid')
    return resault
    #resault.to_csv('D:/luzheng/桌面/student_page_time.csv')

    
def student_dis(df):
    uid = df['uid'].tolist()
    conn = pymysql.connect(host='***',port=3306,user='***',
                           passwd='***',db='talk',charset='utf8')
    cur = conn.cursor(pymysql.cursors.SSCursor)
    cur.execute('''select id, sex, age, is_buy, city, is_staff, birthday from user
                   where id in ('''+",".join(uid)+''')
                ''')
    user_data = [[str(row[0]), row[1], row[2], 
                  row[3], row[4], row[5], row[6]] for row in cur]
        
    df_user = pd.DataFrame(user_data, columns=['uid', 'sex', 'age', 
                                               'buy_type', 'city','staff_type','birthday'])
    df_stu = pd.merge(df, df_user, how='left', on='uid')
    
    df_stu['age'].fillna(0, inplace = True)
    df_stu['sex'].fillna('-', inplace = True)
    df_stu['city'].fillna('未知', inplace = True)
    df_stu['city'] = df_stu['city'].replace('', '未知')
    df_stu['birthday'] = pd.to_datetime(df_stu['birthday'], errors='coerce')
    df_stu['age_new'] = pd.datetime.now().year - df_stu['birthday'].dt.year
    
    #return df_stu
    df_stu.to_csv('D:/luzheng/桌面/student_data_2.csv', encoding='utf-8')
    

#student_resault
#resault_student(df_set())
#student_dis(resault_student(df_set()))

In [4]:
import pandas as pd
import numpy as np

#apriori with user_analysis

def df_set():
    f = pd.read_csv('D:\\luzheng\\桌面\\page_step_new_2.csv', sep='*', 
                header = None, iterator = True, encoding = 'latin1', dtype='unicode',
                names = ['uid', 'ref', 'url', 'click_date', 'time_page', 'step'])
    df = f.get_chunk(100000) #total rows: 14659399
    #df[['time_page', 'step']] = df[['time_page', 'step']].astype(float)
    #df['time_page'] = df['time_page'].replace(-1, 0)
    return df


def load_data():
    df = df_set()
    ref_url = [[row['ref'], row['url']]
               for index, row in df.iterrows()] #遍历df转换2维列表
    return np.array(ref_url)

    
def create_c1(data_set):
    c1 = []
    for row in data_set:
        for col in row:
            if [col] not in c1:           #对列表元素去重
                c1.append([col])
    c1.sort()
    return list(map(frozenset, c1))       #创建不变集合
            

def scan_d(d, c1, minsupport):
    sscnt = {}
    for d_row in d:
        for c_row in c1:
            if c_row.issubset(d_row):        #若c是d的子集
                if c_row not in sscnt: 
                    sscnt[c_row] = 1        #frozenset作为字典键
                else:
                    sscnt[c_row] += 1
                    
    num_d = float(len(d))
    ret_list = []
    support_data = {}
    for key in sscnt:
        support = sscnt[key] / num_d      #计算支持度
        if support >= minsupport:
            ret_list.insert(0, key)
            
        support_data[key] = support
    return ret_list, support_data


def apriori_gen(lk, k):
    ret_list = []
    lenlk = len(lk)
    for i in range(lenlk):
        for j in range(i+1, lenlk):
            l1 = list(lk[i])[:k-2]
            l2 = list(lk[j])[:k-2]
            l1.sort()
            l2.sort()
            if l1 == l2:
                ret_list.append(lk[i] | lk[j])
    return ret_list


def apriori(data_set, minsupport=0.02):
    c1 = create_c1(data_set)
    d = list(map(set, data_set))
    l1, suppdata = scan_d(d, c1, minsupport)
    l = [l1]
    k = 2
    while (len(l[k-2]) > 0):
        ck = apriori_gen(l[k-2], k)
        lk, supk = scan_d(d, ck, minsupport)
        suppdata.update(supk)
        l.append(lk)
        k += 1
    return l, suppdata


def calc_conf(freqset, h, supportdata, brl, minconf=0.5):
    prunedh = []
    for conseq in h:
        conf = supportdata[freqset] / supportdata[freqset - conseq]
        if conf >= minconf:
            print(freqset-conseq, '--->', conseq, 'conf:', conf)
            brl.append((freqset-conseq, conseq, conf))
            prunedh.append(conseq)
    return prunedh


def rules_from_conseq(freqset, h, supportdata, brl, minconf=0.5):
    m = len(h[0])
    if len(freqset) > (m + 1):
        hmp1 = apriori_gen(h, m + 1)
        hmp1 = calc_conf(freqset, hmp1, supportdata, brl, minconf)
        if len(hmp1) > 1:
            rules_from_conseq(freqset, hmp1, supportdata, brl, minconf)
    

def rules(l, suppdata, minconf = 0.5):
    rule_list = []
    for i in range(1, len(l)):
        for freqset in l[i]:
            h1 = [frozenset([item]) for item in freqset]
            if i > 1:
                rules_from_conseq(freqset, h1, suppdata, rule_list, minconf)
            else:
                calc_conf(freqset, h1, suppdata, rule_list, minconf)
    return rule_list
    

l, suppdata = apriori(load_data())
rules = rules(l, suppdata, minconf=0.5)
rules

frozenset({'http://www.51talk.com/reserve/success'}) ---> frozenset({'http://www.51talk.com/reserve/course'}) conf: 0.7077226606538896


[(frozenset({'http://www.51talk.com/reserve/success'}),
  frozenset({'http://www.51talk.com/reserve/course'}),
  0.7077226606538896)]

In [1]:
from user_agents import parse
import re

ua_string = 'Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Mobile/537.36 xiaodurobot'
user_agent = parse(str(ua_string))

user_agent.is_mobile
match = re.match(r'(.*)/\s(\w+)\s.*/\s(.*?)(?:$|\d)', str(user_agent))    
match
#str(user_agent)

<_sre.SRE_Match object; span=(0, 32), match='iPhone / iOS 7.1.2 / xiaodurobot'>

In [3]:
import pandas as pd
from user_agents import parse
import re
import pymysql
import numpy as np

#user_analysis_student

def df_set():
    f = pd.read_csv('D:\\luzheng\\桌面\\page_step_new_4.csv', sep='$', 
                header = None, iterator = True, encoding = 'latin1', dtype='unicode',
                names = ['uid', 'ref', 'url', 'click_date',
                         'click_date_time', 'time_page',
                         'agent', 'step'])
    
    df = f.get_chunk(201070) #total rows: 201070
    df[['time_page', 'step']] = df[['time_page', 'step']].astype(float)
    df['time_page'] = df['time_page'].replace(-1,0)
       
    for index, row in df.iterrows():
        user_agent = parse(str(row['agent']))
        agent = re.match(r'(.*)/(.*)/(.*)', str(user_agent))
        df.loc[index, 'first_device_type'] = agent.group(1)
        df.loc[index, 'first_system'] = agent.group(2)
        df.loc[index, 'first_browser'] = agent.group(3)
    return df


def resault_student(df):
    df.drop('agent', axis=1)
    time_d = df['time_page'].groupby(df['uid']).sum().reset_index()
    step_d = df['step'].groupby(df['uid']).sum().reset_index()
    
    url_d = []
    for k, v in df.groupby(df['uid']):
        if len(v['url'].tolist()) == 1:
            url_d.append([k, '-'])
        else:
            url_d.append([k, v['url'].tolist()[1]])
            
    time_df = pd.DataFrame(time_d)
    step_df = pd.DataFrame(step_d)
    url_df = pd.DataFrame(url_d, columns=['uid', 'url_se'])
    
    step_df.drop(['uid'], axis=1, inplace = True)
    time_step_df = pd.concat((time_df, step_df), axis=1)
    
    cols = ['ref', 'url', 'click_date', 
            'click_date_time', 'agent',
            'time_page', 'step']
    
    df.drop(cols, axis=1, inplace=True)
    df_joined = pd.merge(df, time_step_df, how='left', on='uid')
    df_joined = df_joined.drop_duplicates(['uid'])
    df_all = pd.merge(df_joined, url_df, how='left', on='uid')
    #return df_joined
    return df_all


def student_dis(df):
    uid = df['uid'].tolist()
    conn = pymysql.connect(host='***',port=3306,user='***',
                           passwd='***',db='talk',charset='utf8')
    cur = conn.cursor(pymysql.cursors.SSCursor)
    cur.execute('''select id, sex, age, is_buy, city, is_staff, birthday from user
                   where id in ('''+",".join(uid)+''')
                ''')
    user_data = [[str(row[0]), row[1], 
                  row[2], row[3], 
                  row[4], row[5], 
                  row[6]] for row in cur]
        
    df_user = pd.DataFrame(user_data, columns=['uid', 'sex', 'age', 
                                               'buy_type', 'city',
                                               'staff_type','birthday'])
    
    df_stu = pd.merge(df, df_user, how='left', on='uid')
    
    df_stu['age'].fillna(0, inplace = True)
    df_stu['sex'].fillna('-', inplace = True)
    df_stu['city'].fillna('未知', inplace = True)
    df_stu['city'] = df_stu['city'].replace('', '未知')
    df_stu['birthday'] = pd.to_datetime(df_stu['birthday'], errors='coerce')
    df_stu['age_new'] = pd.datetime.now().year - df_stu['birthday'].dt.year    
    df_stu.drop(['age','staff_type','birthday'], axis=1, inplace=True)
    #return df_stu
    df_stu.to_csv('D:/luzheng/桌面/student_data_5.csv', encoding='utf-8')


student_dis(resault_student(df_set()))

In [90]:
import pandas as pd
from user_agents import parse
import re
import pymysql
import numpy as np
from collections import Counter

##page_rate_付费模型

def df_set():
    f = pd.read_csv('D:\\luzheng\\桌面\\page_step_new_4.csv', sep='$', 
                header = None, iterator = True, encoding = 'latin1', dtype='unicode',
                names = ['uid', 'ref', 'url', 'click_date',
                         'click_date_time', 'time_page',
                         'agent', 'step'])
    df = f.get_chunk(201070) #total rows: 201070
    
    df['hour']= pd.to_datetime(df['click_date_time']).apply(lambda x: x.strftime('%H'))
    df['day_of_week'] = pd.to_datetime(df['click_date_time']).dt.dayofweek.apply(dayofweek)
    
    for index, row in df.iterrows():
        user_agent = parse(str(row['agent']))
        agent = re.match(r'(.*)/\s+(\w+)\s+.*/\s(.*?)(?:$|\d)', str(user_agent))
        df.loc[index, 'first_system'] = agent.group(2)
        df.loc[index, 'first_browser'] = agent.group(3)
    
    drop_col = [col for col in df.columns 
                  if col not in ['uid', 'url','hour',
                                 'day_of_week', 'first_system', 'first_browser']]
    df.drop(drop_col, axis=1, inplace=True)
    return df


def dayofweek(x):
    days = {0: '周一', 1: '周二', 2: '周三',
            3: '周四', 4: '周五', 5: '周六', 
            6: '周日'}
    return days[x]


def rate(grouped):
    buy_count = grouped['buy_type'].apply(lambda x: 1 if x == 1 else 0).sum()
    free_count = grouped['buy_type'].apply(lambda x: 1 if x == 0 else 0).sum()
    all_count = buy_count + free_count
    rate = buy_count / all_count
    return rate, all_count


def student_dis(df):
    uid = df['uid'].tolist()
    conn = pymysql.connect(host=***',port=3306,user='***',
                           passwd='***',db='talk',charset='utf8')
    cur = conn.cursor(pymysql.cursors.SSCursor)
    cur.execute('''select id, sex, age, is_buy, city, is_staff, birthday from user
                   where id in ('''+",".join(uid)+''')
                ''')
    user_data = [[str(row[0]), row[1], 
                  row[2], row[3], 
                  row[4], row[5], 
                  row[6]] for row in cur]
        
    df_user = pd.DataFrame(user_data, columns=['uid',  'sex', 
                                               'age',  'buy_type', 
                                               'city', 'staff_type', 
                                               'birthday'])
    df_all = pd.merge(df, df_user, how='left', on='uid')
    
    df_all['buy_type'] = df_all['buy_type'].replace('free', 0)
    df_all['buy_type'] = df_all['buy_type'].replace('buy', 1)
    
    return df_all


def rate_count(df_all):
    detail = {}
    user_duplicates = []
    for k, v in df_all.groupby(df_all['url']):
        for uid, btype in zip(v['uid'], v['buy_type']):
            detail.setdefault(k,{})[uid] = btype
            
    for url in detail:
        all_count = len(detail[url])
        buy_count = []
        free_count = []
        for uid in detail[url]:
            if detail[url].get(uid) == 1:
                buy_count.append(uid)
            else:
                free_count.append(uid)
        user_duplicates.append([url, len(buy_count) / all_count, all_count])
    df_all_new = pd.DataFrame(user_duplicates, columns = ['url', 'pay_rate', 'all_count'])
    #df_all_new = df_all.groupby(['url']).apply(rate).reset_index(name='rate')
    #df_all_new['all_count'] = df_all_new['rate'].apply(lambda x: x[1])
    #df_all_new['rate'] = df_all_new['rate'].apply(lambda x: x[0])
    df_sort = df_all_new.sort_values(['pay_rate'], ascending=False)
    #return df_all[['uid','buy_type']].ix[df.url == 'http://www.51talk.com/user/update_pwd']
    return df_sort
    #return df_sort.to_csv('D:/luzheng/桌面/page_rate2.csv', encoding='utf-8')

    
def time_count(df_all):
    counter = []
    for k, v in df_all.groupby(df_all['uid']):
        week_max = Counter(v['day_of_week']).most_common(1)[0][0]
        hour_max = Counter(v['hour']).most_common(1)[0][0]
        counter.append([k, week_max, hour_max])
    counter_df = pd.DataFrame(counter, columns=['uid', 'week_max', 'hour_max'])
    return counter_df

    
def statics(grouped):
    if grouped.count() == 0:
        return {'mean_rate': 0, 
                'total': 0}
    else:
        return {'mean_rate':grouped.sum()/ grouped.count(), 
                'total': grouped.sum()}

    
def rate_output():
    df = student_dis(df_set())
    df_sort = rate_count(df)
    df_all = pd.merge(df, df_sort, how='left', on='url')

    rate = df_all['pay_rate'].groupby(df_all['uid']).apply(statics).unstack()
    normal = df_all.groupby(['uid', 'buy_type'],as_index=False).count()

    normal.drop([col for col in normal.columns 
                      if col not in ['uid','buy_type']], axis=1, inplace=True)
    normal_joined = normal.join(rate, how='left', on=['uid'])
    return normal_joined
    #normal_joined.to_csv('D:/luzheng/桌面/student_data_6.csv', encoding='utf-8')

    
def time_output():
    df = student_dis(df_set())
    normal = df.groupby(['uid', 'buy_type', 
                         'first_system', 'first_browser'],as_index=False).count()
    normal.drop([col for col in normal.columns 
                       if col not in ['uid', 'buy_type', 
                                      'first_system', 'first_browser']], axis=1, inplace=True)
    
    df_time = time_count(df)
    df_all = pd.merge(normal, df_time, how='left', on='uid')
    return df_all
    #df_all.to_csv('D:/luzheng/桌面/student_data_7.csv', encoding='utf-8')

    
df = time_output()
df = df.drop(['buy_type'], axis=1)
df_all = pd.merge(df, rate_output(),how='left', on='uid')
df_dup = df_all.drop_duplicates(['uid'])
df_dup.to_csv('D:/luzheng/桌面/student_data_7.csv', encoding='utf-8')

In [23]:
import pandas as pd
from user_agents import parse
import re
import pymysql
import numpy as np
from collections import Counter

##page_rate_出席模型

def df_set():
    f = pd.read_csv('D:\\luzheng\\桌面\\page_step_new_6.csv', sep='$', 
                header = None, iterator = True, encoding = 'latin1', dtype='unicode',
                names = ['uid', 'ref', 'url', 'click_date',
                         'click_date_time', 'time_page',
                         'agent', 'step', 'type'])
    df = f.get_chunk(458564) #total rows: 458564
    return df
    

def dayofweek(x):
    days = {0: '周一', 1: '周二', 2: '周三',
            3: '周四', 4: '周五', 5: '周六', 
            6: '周日'}
    return days[x]


def system(x):
    user_agent = parse(str(x))
    agent = re.match(r'(.*)/\s+(\w+)\s+.*/\s(.*?)(?:$|\d)', str(user_agent))
    system = agent.group(2)
    return system


def browser(x):
    user_agent = parse(str(x))
    agent = re.match(r'(.*)/\s+(\w+)\s+.*/\s(.*?)(?:$|\d)', str(user_agent))
    browser = agent.group(3)
    return browser


def date_clean(df):
    df['hour']= pd.to_datetime(df['click_date_time']).apply(lambda x: x.strftime('%H'))
    df['day_of_week'] = pd.to_datetime(df['click_date_time']).dt.dayofweek.apply(dayofweek)
    df['first_system'] = df['agent'].apply(system)
    df['first_browser'] = df['agent'].apply(browser)
    df['type'] = df['type'].replace('fail', 0)
    df['type'] = df['type'].replace('success', 1)
        
    drop_col = [col for col in df.columns
                  if col not in ['uid', 'url', 'hour',
                                 'day_of_week', 'first_system', 
                                 'first_browser', 'type']]
    df.drop(drop_col, axis=1, inplace=True)
    return df


def rate_count(df_all):
    detail = {}
    user_duplicates = []
    for k, v in df_all.groupby(df_all['url']):
        for uid, btype in zip(v['uid'], v['type']):
            detail.setdefault(k,{})[uid] = btype
            
    for url in detail:
        all_count = len(detail[url])
        success_count = []
        fail_count = []
        for uid in detail[url]:
            if detail[url].get(uid) == 1:
                success_count.append(uid)
            else:
                fail_count.append(uid)
        user_duplicates.append([url, len(success_count) / all_count, all_count])
    df_all_new = pd.DataFrame(user_duplicates, columns = ['url', 'join_rate', 'all_count'])
    df_sort = df_all_new.sort_values(['join_rate'], ascending=False)
    return df_sort
    #return df_sort.to_csv('D:/luzheng/桌面/page_rate_chuxi.csv', encoding='utf-8')

    
def time_count(df_all):
    counter = []
    for k, v in df_all.groupby(df_all['uid']):
        week_max = Counter(v['day_of_week']).most_common(1)[0][0]
        hour_max = Counter(v['hour']).most_common(1)[0][0]
        counter.append([k, week_max, hour_max])
    counter_df = pd.DataFrame(counter, columns=['uid', 'week_max', 'hour_max'])
    return counter_df

    
def statics(grouped):
    if grouped.count() == 0:
        return {'mean_rate': 0, 
                'total': 0}
    else:
        return {'mean_rate':grouped.sum()/ grouped.count(), 
                'total': grouped.sum()}

    
def rate_output(df):
    df_sort = rate_count(df)
    df_all = pd.merge(df, df_sort, how='left', on='url')

    rate = df_all['join_rate'].groupby(df_all['uid']).apply(statics).unstack()
    normal = df_all.groupby(['uid', 'type'],as_index=False).count()

    normal.drop([col for col in normal.columns 
                      if col not in ['uid', 'type']], axis=1, inplace=True)
    normal_joined = normal.join(rate, how='left', on=['uid'])
    return normal_joined

    
def time_output(df):
    normal = df.groupby(['uid', 'type', 
                         'first_system', 
                         'first_browser'],as_index=False).count()
    normal.drop([col for col in normal.columns 
                       if col not in ['uid', 'type', 
                                      'first_system', 
                                      'first_browser']], axis=1, inplace=True)
    
    df_time = time_count(df)
    df_all = pd.merge(normal, df_time, how='left', on='uid')
    return df_all


df = date_clean(df_set())
df_time = time_output(df)
df_time = df_time.drop(['type'], axis=1)
df_all = pd.merge(df_time, rate_output(df),how='left', on='uid')
df_dup = df_all.drop_duplicates(['uid'])
df_dup.to_csv('D:/luzheng/桌面/student_data_chuxi.csv', encoding='utf-8')

In [67]:
import pandas as pd
import pymysql
import re
from datetime import datetime,date,timedelta
from ncode import NCode
from collections import Counter
from user_agents import parse

#user_analysis


def df_set():
    f = pd.read_csv('D:\\luzheng\\桌面\\page_step_new.csv', sep='$', 
                header = None, iterator = True, encoding = 'latin1', dtype='unicode',
                names = ['uid', 'ref', 'url', 'click_date',
                         'click_time', 'time_page', 'agent',
                         'step'])
    df = f.get_chunk(1000) #total rows: 14658384
    return df


def df_teacher(df):
    pattern_teacher = re.compile(r'.*/teacher/[a-z]+/[a-z0-9]')
    type_url = []
    for url in df['url']:
        if pattern_teacher.match(url):
            type_url.append('teacher')
        else:
            type_url.append('--')   
    df['type_url'] = type_url
    df_teacher = df[df['type_url']=='teacher']
    return df_teacher


def search_t(df):
    df = df.copy()
    teacher_list = []
    pattern = re.compile(r'.*/teacher/[a-z]+/([a-z0-9]+)')
    for index, row in df.iterrows():
        m = pattern.match(row['url'])
        teacher_list.append(NCode().decode(m.group(1)))
        #teacher_list.append(m.group(1))
    df['t_id'] = teacher_list
    return df


def removeStopWord(string):
    STOP_WORDS = ['省','市','自治区','自治州','自治县','地区','维吾尔','蒙古族','回族','藏族','维吾尔族',
              '苗族','彝族','壮族','布依族','朝鲜族','满族','侗族','瑶族','白族','土家族','哈尼族',
              '哈萨克族','傣族','黎族','僳僳族','佤族','畲族','高山族','拉祜族','水族','东乡族','纳西族',
              '景颇族','柯尔克孜族','土族','达斡尔族','仫佬族','羌族','布朗族','撒拉族','毛南族','仡佬族',
              '锡伯族','阿昌族','普米族','塔吉克族','怒族','乌兹别克族','乌孜别克族','俄罗斯族','鄂温克族',
              '德昂族','保安族','裕固族','京族','塔塔尔族','独龙族','鄂伦春族','赫哲族','门巴族','珞巴族',
              '基诺族']
    
    if len(string) == 0:
        return ''
    string = string.strip()
    for stop_word in STOP_WORDS:
        parts = re.split(stop_word,string)
        string = parts[0]
    return string


def occup(row):
    if row in [1, 2, 5]:
        row = '成人'
    elif row in [3, 4, 6, 7]:
        row = '青少'
    else:
        row = '其他'
    return row


def device_type(row):
    user_agent = parse(str(row))
    pattern = re.compile(r'(.*)/(.*)/(.*)')
    agent = pattern.match(str(user_agent))
    return agent.group(1)


def system(row):
    user_agent = parse(str(row))
    pattern = re.compile(r'(.*)/(.*)/(.*)')
    agent = pattern.match(str(user_agent))
    return agent.group(2)
                      

def browser(row):
    user_agent = parse(str(row))
    pattern = re.compile(r'(.*)/(.*)/(.*)')
    agent = pattern.match(str(user_agent))
    return agent.group(3)


def student_dis(df):
    uid = df['uid'].tolist()
    conn = pymysql.connect(host='116.213.69.48',port=3306,user='data',
                           passwd='Data_0723',db='talk',charset='utf8')
    cur = conn.cursor(pymysql.cursors.SSCursor)
    cur.execute('''select id, sex, age, is_buy, city, is_staff, birthday, occup
                   from user
                   where id in ('''+",".join(uid)+''')
                ''')
    user_data = [[str(row[0]), row[1], 
                  row[2], row[3], 
                  row[4], row[5], 
                  row[6], row[7]] for row in cur]
    
    df_user = pd.DataFrame(user_data, columns=['uid',  'sex', 
                                               'age',  'buy_type', 
                                               'city', 'staff_type',
                                               'birthday', 'occup'])
    
    df_all = pd.merge(df, df_user, how='left', on='uid')
    df_all['age'].fillna(0, inplace = True)
    df_all['sex'].fillna('-', inplace = True)
    df_all['city'].fillna('未知', inplace = True)
    
    df_all['city'] = df_all['city'].apply(removeStopWord)
    df_all['occup'] = df_all['occup'].apply(occup)
    df_all['device_type'] = df_all['agent'].apply(device_type)
    df_all['system'] = df_all['agent'].apply(system)
    df_all['browser'] = df_all['agent'].apply(browser)
    
    df_all['birthday'] = pd.to_datetime(df_all['birthday'], errors='coerce')
    df_all['age_new'] = pd.datetime.now().year - df_all['birthday'].dt.year    
    
    df_all[['time_page', 'step']] = df_all[['time_page', 'step']].astype(float)
    df_all['time_page'] = df_all['time_page'].replace(-1, 0)
    
    df_all.drop(['ref', 'url', 'age', 
                 'staff_type', 'agent',
                 'birthday', 'type_url',
                 'step', 'click_date'], axis=1, inplace=True)
    return df_all

df_t = df_teacher(df_set())
df = search_t(df_t)
df_new = student_dis(df)
detail = []

for k, v in df_new.groupby(['t_id']):
    city = ','.join(str(v) for v in Counter(v['city']).most_common(3))
    sex = ','.join(str(v) for v in Counter(v['sex']).most_common())
    occup = ','.join(str(v) for v in Counter(v['occup']).most_common())
    device_type = ','.join(str(v) for v in Counter(v['device_type']).most_common(3))
    system = ','.join(str(v) for v in Counter(v['system']).most_common(3))
    browser = ','.join(str(v) for v in Counter(v['browser']).most_common(3))
    page_time = v['time_page'].sum()
    detail.append([k, city, sex, occup, 
                   device_type, system, 
                   browser, page_time])

detail_df = pd.DataFrame(detail, columns=['t_id', 'stu_city', 
                                          'stu_sex', 'stu_type', 
                                          'stu_device', 'stu_system',
                                          'stu_browser', 'stu_page_time'])

time_sort = detail_df.sort_values(['stu_page_time'], ascending=False)

time_sort
#time_sort.to_csv('D:/luzheng/桌面/teacher_page_time2.csv')

,t_id,stu_city,stu_sex,stu_type,stu_device,stu_system,stu_browser,stu_page_time
15,18787,"('上饶', 2)","('man', 2)","('青少', 2)","('PC ', 2)","(' Windows 7 ', 2)","(' Chrome 31.0.1650', 2)",292
3,11311,"('北京', 2)","('man', 2)","('青少', 2)","('PC ', 2)","(' Windows XP ', 2)","(' Chrome 31.0.1650', 2)",203
2,10521,"('成都', 2)","('man', 2)","('其他', 2)","('PC ', 2)","(' Windows 10 ', 2)","(' QQ Browser 9.2.5748', 2)",196
27,387,"('北京', 1)","('man', 1)","('成人', 1)","('PC ', 1)","(' Windows 10 ', 1)","(' Chrome 42.0.2311', 1)",173
12,16357,"('上饶', 3)","('man', 3)","('青少', 3)","('PC ', 3)","(' Windows 7 ', 3)","(' Chrome 31.0.1650', 3)",143
7,12119,"('天津', 2)","('man', 2)","('成人', 2)","('PC ', 2)","(' Windows 7 ', 2)","(' Chrome 19.0.1084', 2)",129
31,5439,"('天津', 2)","('man', 2)","('成人', 2)","('PC ', 2)","(' Windows 7 ', 2)","(' Chrome 19.0.1084', 2)",123
16,19625,"('上饶', 1)","('man', 1)","('青少', 1)","('PC ', 1)","(' Windows 7 ', 1)","(' Chrome 31.0.1650', 1)",96
1,10363,"('广州', 7)","('man', 7)","('成人', 7)","('PC ', 7)","(' Windows 8.1 ', 7)","(' Maxthon 4.4.2', 7)",69
0,10102,"('北京', 1)","('woman', 1)","('青少', 1)","('PC ', 1)","(' Windows 7 ', 1)","(' Chrome 31.0.1650', 1)",62
